In [1]:
from transformers import pipeline, AutoTokenizer
import torch

# model_id = "mistralai/Mistral-7B-Instruct-v0.2"
model_id = "Qwen/Qwen2.5-7B-Instruct"
# model_id = "Qwen/Qwen1.5-1.8B-Chat"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# индекс устройства для pipeline: 0 для cuda, -1 для cpu
device_index = 0 if device.type == "cuda" else -1

# токенизатор для чатовской разметки
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

generation_pipeline = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
    device=device_index,
)

2025-11-16 18:57:30.866156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763319451.300845      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763319451.407720      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
system_msg = (
"""
You are an elite senior software engineer and AI coding assistant.
Your mission is to produce the highest-quality code and explanations possible.
Follow these rules strictly:
1. Code Quality
Always write clean, idiomatic, production-ready code.
Use best practices for the chosen language (naming, structure, style).
Never leave unused imports, dead code, or unclear variables.
Prefer clarity over cleverness.
Ensure code is robust and handles edge cases.
2. Structure
When providing code:
Provide full self-contained examples.
Include all imports.
Include comments explaining nontrivial parts.
If multiple files are needed, show them separately.
3. Reasoning & Explanation
Before coding, briefly outline the solution.
Explain the logic clearly and concisely.
If the user request is ambiguous, ask clarifying questions.
Offer improvements or optimizations only if they are directly relevant.
4. Safety & Correctness
Validate user input.
Avoid deprecated methods.
Follow secure coding practices.
Prefer pure functions and modular design when possible.
5. Performance
Choose efficient algorithms and data structures.
Mention time and space complexity when relevant.
If optimization is possible, explain trade-offs.
6. Testing
When useful, provide unit tests.
Ensure tests cover normal cases, edge cases, and failures.
7. Tools & Libraries
Use modern, stable libraries.
Avoid heavy or unnecessary dependencies.
8. User Context Awareness
Adapt solutions to the user’s stack when known.
If the user works with ML/AI, provide GPU-friendly and efficient code.
If working with API/web, follow industry standards.
9. Formatting
Code should use consistent spacing, indentation, and style.
Do not mix tabs and spaces.
Use monospace blocks for code.
No trailing comments outside code blocks.
"""
)

user_msg = (
"""
Write a Python function to calculate the metric medianAPE = mediani(|^yi−yi|/yi)

where yi is the correct answer and ^yi are the prediction models.


"""
)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_msg},
]
# готовим prompt через чат шаблон
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

outputs = generation_pipeline(
    prompt,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    return_full_text=False,  # вернуть только продолжение, без промпта
)

answer = outputs[0]["generated_text"].strip()
print(answer)

To calculate the Median Absolute Percentage Error (medianAPE), we need to follow these steps:

1. Compute the absolute percentage error (APE) for each pair of actual values (`yi`) and predicted values (`^yi`).
2. Find the median of these APEs.

Let's break down the process and then implement it in Python.

### Steps:
1. **Compute APE**: For each pair `(yi, ^yi)`, compute the absolute percentage error using the formula \( \text{APE} = \left| \frac{\hat{y}_i - y_i}{y_i} \right| \).
2. **Find the Median**: Once we have the list of APEs, find the median value.

### Implementation:

```python
import numpy as np

def calculate_medianAPE(y_true, y_pred):
    """
    Calculate the Median Absolute Percentage Error (medianAPE).

    Parameters:
    y_true (list or np.array): Actual values.
    y_pred (list or np.array): Predicted values.

    Returns:
    float: The median absolute percentage error.
    """
    # Ensure inputs are numpy arrays for easier manipulation
    y_true = np.array(y_true